In [34]:
from keras.layers import Dense,Activation
import pandas as pd
from keras.models import Sequential

In [66]:
trustdb=pd.read_csv('train/trustdb_gaussian.csv')
#train data with features frequenct, intimacy and recency (gaussian normalized i.e. (x-u)/sd)
X=trustdb.drop(['TRUST',],axis=1).values
#outputs
Y=trustdb['TRUST'].values
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
Y = to_categorical(Y, num_classes = 4)

In [58]:
#defining our 2 hidden layer perceptron
model= Sequential()
model.add(Dense(3,input_dim=3))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
#should get 99.88 % accuracy
model.fit(X, Y, validation_split=0.33, epochs=10, batch_size=10,shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 3)                 12        
_________________________________________________________________
activation_17 (Activation)   (None, 3)                 0         
_________________________________________________________________
dense_50 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_18 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_51 (Dense)             (None, 4)                 24        
_________________________________________________________________
activation_19 (Activation)   (None, 4)                 0         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 8982 

In [43]:
#saves our model for 4 class trust
model.save('models/trust_4class_mlp.h5')

In [81]:
#loads our model for 4 class trust
from keras.models import load_model
model = load_model('models/trust_4class_mlp.h5')
#since the values are one hot encoded, taking argmax indicates the index value of the class it belongs to
#also we have to reshape the array since the first parameter is unspecified
np.argmax(model.predict(X[7760].reshape(1,3)))

7

In [4]:
import os
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os.path as osp

saved_model_path='models/trust_4class_mlp'
path_to_model='models/trust_4class_mlp.h5'
model = load_model(path_to_model)
nb_classes = 1 # The number of output nodes in the model
prefix_output_node_names_of_final_network = 'output_node'

K.set_learning_phase(0)

pred = [None]*nb_classes
pred_node_names = [None]*nb_classes
for i in range(nb_classes):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

sess = K.get_session()
output_fld = 'export_tf /'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = saved_model_path + '.pb'
output_graph_suffix = '_inference'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))


output nodes names are:  ['output_node0']
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
saved the constant graph (ready for inference) at:  tensorflow_model/trust_4class_mlp.pb


In [5]:
def print_graph_nodes(filename):
    import tensorflow as tf
    g = tf.GraphDef()
    g.ParseFromString(open(filename, 'rb').read())
    print()
    print(filename)
    print("=======================INPUT=========================")
    print([n for n in g.node if n.name.find('input') != -1])
    print("=======================OUTPUT========================")
    print([n for n in g.node if n.name.find('output') != -1])
    print("===================KERAS_LEARNING=====================")
    print([n for n in g.node if n.name.find('keras_learning_phase') != -1])
    print("======================================================")
    print()
print_graph_nodes('export_tf /trust_4class_mlp.pb')


tensorflow_model/trust_4class_mlp.pb
=======================INPUT=========================
[name: "dense_46_input_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 3
      }
    }
  }
}
]
=======================OUTPUT========================
[name: "output_node0"
op: "Identity"
input: "strided_slice"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
]
===================KERAS_LEARNING=====================
[]

